In [ ]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=021ece33cad5b62117496c97252a7c50bb77227e889f82141ac471910306550c
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [ ]:
import wikipedia
import tensorflow as tf
import numpy as np
import re

# Define topics
topics = [
    'Data Science',
    'Artificial Intelligence',
    'Neural Networks',
    'Natural Language Processing',
    'Deep Learning',
    'Reinforcement Learning',
    'Big Data',
    'Statistics',
    'Python Programming Language',
    'Politics',
    'Business',
    'Physics',
    'Mathematics',
    'Tesla Motors',
    'OpenAI',
    'C++ programming Language',
    'Military'
]

topic_contents = []
for topic in topics:
    try:
        content = wikipedia.page(topic).content

        if content:
            #Limit content to 200 words (it can be changed as per the need)
            limited_content = ' '.join(content.split()[:200])
            topic_contents.append(limited_content)
        else:
            print(f"Empty content for {topic}. Skipping.")
    except wikipedia.exceptions.PageError:
        print(f"Page for {topic} not found.")
    except wikipedia.exceptions.DisambiguationError:
        print(f"Ambiguous page for {topic}.")

#Combine and clean text data
combined_content = ' '.join(topic_contents)
cleaned_text = re.sub(r'[^a-zA-Z\s]', '', combined_content).lower()

# Tokenize
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts([cleaned_text])
tokens = tokenizer.texts_to_sequences([cleaned_text])[0]

#Create sequences of input and output data
seq_length = 30  #Limiting sequence length to 30 words
sequences = []
for i in range(0, len(tokens) - seq_length, 1):
    sequences.append(tokens[i:i + seq_length + 1])

sequences = np.array(sequences)

#Shuffle the sequences
np.random.shuffle(sequences)

#Split sequences into input and target
X = sequences[:, :-1]
y = sequences[:, -1]

print("Total number of tokens:", len(tokens))
print("Vocabulary size:", len(tokenizer.word_index))

#Check data validity
if sequences.any():
    print("Dataset shape:", sequences.shape)
else:
    print("The dataset is empty or contains invalid data.")


/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Ambiguous page for Physics.
Total number of tokens: 3136
Vocabulary size: 1140
Dataset shape: (3106, 31)


In [ ]:
vocab_size = len(tokenizer.word_index) + 1  #Vocabulary size (+1 for padding)
embedding_dim = 256
rnn_units = 512

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=seq_length),
    tf.keras.layers.LSTM(256, return_sequences=True, activation='tanh'),
    tf.keras.layers.LSTM(512, return_sequences=True, activation='tanh'),
    tf.keras.layers.LSTM(512, activation='tanh'),
    tf.keras.layers.Dense(vocab_size, activation='softmax')])

#Compile the model
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

#Model Training
model.fit(X, y, batch_size=64, epochs=20, validation_split=0.2)


Epoch 1/20
39/39 [==============================] - 93s 2s/step - loss: 6.5857 - accuracy: 0.0475 - val_loss: 6.4994 - val_accuracy: 0.0450
Epoch 2/20
39/39 [==============================] - 87s 2s/step - loss: 6.1675 - accuracy: 0.0419 - val_loss: 6.6619 - val_accuracy: 0.0450
Epoch 3/20
39/39 [==============================] - 88s 2s/step - loss: 6.0767 - accuracy: 0.0495 - val_loss: 6.8237 - val_accuracy: 0.0450
Epoch 4/20
39/39 [==============================] - 91s 2s/step - loss: 6.0842 - accuracy: 0.0471 - val_loss: 6.8993 - val_accuracy: 0.0305
Epoch 5/20
39/39 [==============================] - 90s 2s/step - loss: 6.0298 - accuracy: 0.0447 - val_loss: 6.9118 - val_accuracy: 0.0450
Epoch 6/20
39/39 [==============================] - 90s 2s/step - loss: 6.0029 - accuracy: 0.0366 - val_loss: 6.9656 - val_accuracy: 0.0450
Epoch 7/20
39/39 [==============================] - 90s 2s/step - loss: 5.9732 - accuracy: 0.0491 - val_loss: 7.0267 - val_accuracy: 0.0450
Epoch 8/20
39/39 [==

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 30, 256)           292096    
                                                                 
 lstm_6 (LSTM)               (None, 30, 256)           525312    
                                                                 
 lstm_7 (LSTM)               (None, 30, 512)           1574912   
                                                                 
 lstm_8 (LSTM)               (None, 512)               2099200   
                                                                 
 dense_2 (Dense)             (None, 1141)              585333    
                                                                 
Total params: 5076853 (19.37 MB)
Trainable params: 5076853 (19.37 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
#Function to generate text using the trained model
def generate_text_with_temperature(topic, next_words, temperature=1.0):
    generated_text = topic.lower()
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([generated_text])[0]
        token_list = tf.keras.preprocessing.sequence.pad_sequences([token_list], maxlen=seq_length, padding='pre')

        predicted = model.predict(token_list, verbose=0)[-1]  # Last prediction
        predicted = np.log(predicted) / temperature
        exp_preds = np.exp(predicted)
        predicted = exp_preds / np.sum(exp_preds)

        predicted_index = np.random.choice(len(predicted), p=predicted)
        output_word = tokenizer.index_word.get(predicted_index, "")

        generated_text += " " + output_word
    return generated_text

#Prompt the model for information related to a topic with temperature control
topic = input('Ask me from the listed topics.....\n')
generated_info = generate_text_with_temperature(topic, next_words=50, temperature=1)
print(generated_info)


Ask me from the listed topics.....
Tesla Motors
tesla motors diverse internal it in the of greek underlying naturallanguage forms may intelligence identify was to cannot elon premise as a and concerns games led refers as business knowledge nature nonprofit or and andin fundamental such subfield goal naturallanguage data usually a not history the range swarm programs machine learning datadriven


In [ ]:
from language_tool_python import LanguageTool

# Assuming `generated_text` contains the output generated by your model
generated_text = "Your generated text here..."

# Create a LanguageTool instance
tool = LanguageTool('en-US')

# Check and correct grammar errors in the generated text
matches = tool.check(generated_text)
corrected_text = tool.correct(generated_info)

# Print corrected text
print("Original Text:")
print(generated_info)
print("\nCorrected Text:")
print(corrected_text)

Original Text:
tesla motors diverse internal it in the of greek underlying naturallanguage forms may intelligence identify was to cannot elon premise as a and concerns games led refers as business knowledge nature nonprofit or and andin fundamental such subfield goal naturallanguage data usually a not history the range swarm programs machine learning datadriven

Corrected Text:
Tesla motors diverse internal it in the of Greek underlying natural language forms may intelligence identify was to cannot Elon premise as a and concerns games led refers as business knowledge nature nonprofit or and an din fundamental such subfield goal natural language data usually a not history the range swarm programs machine learning data driven


In [ ]:
import pickle as pe
tokenizer_file = 'tokenizer_model.pkl'
with open(tokenizer_file, 'wb') as token_file:
    pe.dump(tokenizer, token_file)

In [ ]:
#Save the model
model.save("Chat_pedia_model.h5")

#Load the model
#loaded_model = tf.keras.models.load_model("Chat_pedia_model.h5")


In [ ]:
!pip install language-tool-python

In [7]:
import pickle

with open('tokenizer_model.pkl', 'rb') as file:
    tokenizer1 = pickle.load(file)


In [12]:
import json

tokenizer_config = tokenizer1.to_json()

with open('tokenizer_config.json', 'w') as json_file:
    json.dump(tokenizer_config, json_file)

In [9]:
tokenizer1